In [4]:
pip install transformers datasets torch

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached xxhash-3.5.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata (13 kB)
  Using cached multidict-6.1.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata (5.0 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.1/32.1 MB 6.7 MB/s eta 0:00:0000:0100:01m
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached xxhash-3.5.0-cp310-cp310-macosx_10_9_x86_64.whl (31 kB)
Using cached frozenlist-1.5.0-cp310-cp310-macosx_10_9_x86_64.whl (54 kB)
Using cached multidict-6.1.0-cp310-cp310-macosx_10_9_x86_64.whl (29 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninst

In [10]:
pip install --upgrade --force-reinstall huggingface_hub

  Using cached huggingface_hub-0.28.1-py3-none-any.whl.metadata (13 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached huggingface_hub-0.28.1-py3-none-any.whl (464 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.14
    Uninstalling urllib3-1.26.14:
      Successfully uninstalled urllib3-1.26.14
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninsta

In [1]:
import numpy as np
from transformers import AlbertTokenizer, AlbertForQuestionAnswering # import AlBERT model
from datasets import load_dataset

In [ ]:
data = load_dataset('squad')

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})
